In [580]:
import pandas as pd
import openpyxl
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from scipy.stats import describe, ttest_1samp
from pathlib import Path
from openpyxl import load_workbook
import os

## Read the data

In [581]:
# rfr file, rm file
file_name_rfr = r"..\..\Sray\original dirty data\New_rfr.xlsx"
file_name_rm = r"..\..\Sray\original dirty data\New_rm.xlsx"

#get the sorted fin file

file_name_str = r"..\..\Sray\Analysis_bottom\AU\AU_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis_btm\CN\CN_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis_btm\HK\HK_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis_btm\MY\MY_Bloomberg_fin_data_portfolio.xlsx"
# file_name_str = r"..\Sray\Analysis_btm\SG\SG_Bloomberg_fin_data_portfolio.xlsx"



indicator = 'e'
# indicator = 's'
# indicator = 'g'
# indicator = 'esg'
# indicator = 'esg_value'
# indicator = 'value'
# indicator = 'historical_market_cap'
# indicator = 'roa'
# indicator = 'vol_180d'
## indicator = 'return_com_eqy'

file_name = Path(file_name_str).stem
fin_file = pd.ExcelFile(file_name_str)
rfr_file = pd.ExcelFile(file_name_rfr)
rm_file = pd.ExcelFile(file_name_rm)

findata_sheet = pd.read_excel(
    fin_file, sheet_name="main", index_col=0, header=0)

porfolio_selection = pd.DataFrame(findata_sheet)


porfolio_selection.head()


,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
64,BEACH ENERGY LTD,BPT AU Equity,1,2017,2017-12-31,63.715923,55.934765,56.559856,78.597229,3163.662269,...,-0.231913,26.615385,-46.902965,143.144459,0.513595,0.244650,-16.640570,25.681782,1.426979,31.980287
64,BEACH ENERGY LTD,BPT AU Equity,1,2018,2018-12-31,63.917110,52.823921,53.990326,84.858521,2851.980723,...,0.150097,162.555556,-42.855693,144.975789,1.416866,0.450774,-6.930570,31.560391,0.607143,32.183942
64,BEACH ENERGY LTD,BPT AU Equity,1,2019,2019-12-31,58.867317,38.447598,53.204353,84.858521,2045.579562,...,-0.335546,-33.146067,-65.040082,199.914059,0.159539,-0.114482,-9.693295,23.544837,0.995420,29.376417
64,BEACH ENERGY LTD,BPT AU Equity,1,2020,2020-12-31,58.867317,38.447598,53.204353,84.858521,2045.579562,...,-0.236037,-29.787234,-19.155449,78.119710,-0.136096,-0.159078,-14.615757,23.006446,0.995423,29.354120
64,BEACH ENERGY LTD,BPT AU Equity,1,2021,2021-12-31,56.443012,31.168831,53.204353,84.858521,1658.317491,...,-0.177347,110.472973,-36.657296,246.943174,0.595806,0.102629,-6.417029,22.538936,0.673037,28.272821


## Sorting the data by firm name and then date

In [582]:
portfolio_selection_sorted = porfolio_selection.sort_values(
    by=['firm_name', 'date'])
portfolio_selection_sorted.head(10)

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,sp_z,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value
firm no,,,,,,,,,,,,,,,,,,,,,
346,29METALS LTD,29M AU Equity,2,2017,2017-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-2.478196,45.795635,NaN,NaN,-9.662817,37.771376,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2018,2018-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-16.507780,100.808472,NaN,NaN,-8.171819,41.974506,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2019,2019-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-27.039477,118.522743,NaN,NaN,-9.197116,38.048781,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2020,2020-12-31,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.587213,52.219266,NaN,NaN,-9.134804,29.156100,NaN,NaN
346,29METALS LTD,29M AU Equity,2,2021,2021-12-31,NaN,NaN,NaN,NaN,NaN,...,0.468829,-8.493156,-21.447492,107.377879,0.120643,0.385934,4.190439,35.369566,0.147600,NaN
346,29METALS LTD,29M AU Equity,2,2022,2022-12-30,32.863895,5.708245,29.201935,63.576160,166.691793,...,-0.015473,-11.694764,-34.498056,147.401152,0.154702,0.459116,0.098781,38.126217,0.244247,16.661506
346,29METALS LTD,29M AU Equity,2,2023,2023-01-31,NaN,NaN,NaN,NaN,NaN,...,0.024564,-11.396429,-30.772953,125.052021,0.154948,0.501355,0.098781,38.126217,0.244247,NaN
75,A2B AUSTRALIA LTD,A2B AU Equity,3,2017,2017-12-31,26.798109,0.906071,12.575574,66.767006,11.394363,...,-0.219742,4.433679,10.298060,30.841917,-0.190143,0.475697,2.450075,10.992115,0.101175,13.636903
75,A2B AUSTRALIA LTD,A2B AU Equity,3,2018,2018-12-31,26.285082,0.000000,11.940750,66.767006,0.000000,...,-0.229590,3.598049,7.360237,29.933450,-0.125685,-0.321829,4.425666,6.625187,-0.043375,12.981626


## Ranking stocks percentile
| Country/Region | Percentile chosen (Top(%))|
| --- | --- |
| Australia (AU)| 20 |
| China (CN)| 20 |
| Malaysia (SG)| 20 |
| Hong Kong (SG)| 25 |
| Singapore (SG)| 25 |

## Excess Return
At the end of each month, compute the cross-sectional portfolio return of esg_top20. For example, at the end of August, after selecting the top 20% stocks based on August ESG scores, compute the raw returns of each stocks in the high-rated portfolio (the top 20%) by the following equation: $$ret_{i}  = P_{1}/P_{0} -1 $$(for example, after ranking stocks at the end of Aug, calculate the individual returns by using $$(Sept-price/Aug-price) – 1)$$ Next, compute excess returns of each stocks: $$r_{i} = ret_{i} – r_{f}$$ where r_i is the excess return of stock i and r_f is the respective country’s monthly risk-free rate. So now we have r_1, …, r_n where n = number of stocks in the high-rated portfolio. Next, we equal-weight (average) all the r in the esg_top 20 and find the portfolio mean (denoted as μ_t) for each month. This generates the High portfolio excess return (equal weight) in the Excel file for each month. Scroll down the Excel file, compute the mean of all the months’ excess returns, and test for significance (H0: mean > 0). Indicate the t-stat value.

In [583]:
"""
change data format
"""

portfolio_selection_sorted['date'] = portfolio_selection_sorted['date'].astype(
    'datetime64')
portfolio_selection_sorted.dtypes


firm_name                        object
tkr                              object
sec_no                            int64
year                              int64
date                     datetime64[ns]
esg                             float64
e                               float64
s                               float64
g                               float64
es                              float64
eg                              float64
sg                              float64
esg_cubic                       float64
px                              float64
fcf                             float64
pe                              float64
beta                            float64
roa                             float64
vol_180d                        float64
sales                           float64
ret                             float64
historical_market_cap           float64
ep                              float64
ep_sec_mean                     float64
ep_sec_sd                       float64


In [584]:
# new_companies_data.to_excel("SG_Bloomberg_fin_data_portfolio.xlsx", sheet_name="main")
quantile_rank = 0
quantile_path = ''
quantile_folder = ''


quantile_folder = "..\..\Sray\Analysis_top\\" + file_name[:2] + "\\" + indicator
quantile_path = quantile_folder + "\\" + file_name[:2] +"_portfolio_" + indicator + "_quantile_rank.xlsx"
top_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_top20_second_highest.xlsx"
reporting_path = quantile_folder + "\\" + file_name[:2] + "_" + indicator + "_int_reporting_second_highest.xlsx"
rfr_sheet = pd.read_excel(rfr_file, sheet_name= file_name[:2] )
rm_sheet = pd.read_excel(rm_file, sheet_name= file_name[:2] )

try:
    os.makedirs(quantile_folder)
except FileExistsError:
    # directory already exists
    pass

if file_name[:2].lower() == 'AU'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'CN'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'HK'.lower():
    quantile_rank = 5

elif file_name[:2].lower() == 'MY'.lower():
    quantile_rank = 4

elif file_name[:2].lower() == 'SG'.lower():
    quantile_rank = 4
    

In [585]:
rfr_sheet = rfr_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rfr_yr': 'rfr'}, axis=1)
rfr_sheet

,year,rfr
0,2017,0.018870
1,2018,0.000634
2,2019,0.010361
3,2020,0.000686
4,2021,0.000571


In [586]:
rm_sheet = rm_sheet.rename({'Year': 'year'.lower(), file_name[:2]+'_rm_yr': 'rm'}, axis=1)
rm_sheet

,year,rm
0,2017,0.094229
1,2018,-0.111802
2,2019,0.096939
3,2020,-0.094601
4,2021,0.136392


In [587]:
rfr_rm_sheet = pd.merge(rfr_sheet, rm_sheet, how='left', on='year')
rfr_rm_sheet

,year,rfr,rm
0,2017,0.018870,0.094229
1,2018,0.000634,-0.111802
2,2019,0.010361,0.096939
3,2020,0.000686,-0.094601
4,2021,0.000571,0.136392


In [588]:
"""
gets all the companies, dropping the empty columns
"""
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates()

#set the data to a list
companies = companies.tolist()


#ensure the number of companies exist
len(companies)


334

In [589]:
# new_companies_data.to_excel("SG_data_2.28_esgj3mean.xlsx")

### Rank the companies 

In [590]:
companies_sray = []
all_quantile_rank = []

initial_year = 2017
stop_year = 2023


for year in range(initial_year, stop_year+1, 1):

        # temp_dataframe = fin_data.loc[(fin_data['sec_no']==k+1) & (fin_data['date']>=start_date) & (fin_data['date']<=end_date)].copy()
        temp_dataframe = portfolio_selection_sorted.loc[(
            portfolio_selection_sorted['year'] == year)].copy()

        
        """
        sort by the relative value and then divide them to relative division
        """

        temp_dataframe.sort_values(by=[indicator], inplace=True)

        temp_dataframe['quantile_rank'] = pd.qcut(temp_dataframe[indicator],
                                                    q=quantile_rank, labels=False, duplicates='drop')
        

        temp_dataframe_quantile = temp_dataframe.copy()

        companies_sray.append(temp_dataframe)

        all_quantile_rank.append(temp_dataframe_quantile)


In [591]:
all_sort_companies_data = pd.concat(all_quantile_rank)

all_sort_companies_data.to_excel(
    quantile_path, sheet_name="main")


In [592]:
sort_companies_data = pd.concat(companies_sray)
sort_companies_data.head()

,firm_name,tkr,sec_no,year,date,esg,e,s,g,es,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
firm no,,,,,,,,,,,,,,,,,,,,,
316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,0.0,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,0.0,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,0.0,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,0.0,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,0.0,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


In [593]:
all_sort_companies_data = all_sort_companies_data.reset_index()
all_sort_companies_data = pd.merge(all_sort_companies_data, rfr_rm_sheet, how='left', on='year')
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('ret')+1, 'rfr', all_sort_companies_data.pop('rfr'))
all_sort_companies_data.insert(all_sort_companies_data.columns.get_loc('rfr')+1, 'rm', all_sort_companies_data.pop('rm'))

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


In [594]:
df_sec_company_count_sum = []
df_sec_company_count = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].count())
df_sec_company_count = df_sec_company_count.rename({'tkr':'tkr_count'}, axis=1)
df_sec_company_nunique = pd.DataFrame(sort_companies_data.groupby(['quantile_rank','year'])['tkr'].nunique())
df_sec_company_nunique = df_sec_company_nunique.rename({'tkr':'tkr_nunique'}, axis=1)

df_sec_company_count = pd.merge(df_sec_company_count, df_sec_company_nunique, on=['quantile_rank','year'])

In [595]:
df_sec_company_count_sum = list()
# for rank in range (0, quantile_rank, 1):
groups = df_sec_company_count.groupby(level='quantile_rank')

    # loop through the groups to create separate dataframes
dfs = {}
for name, group in groups:
    dfs[name] = group.copy()
        
    # display the separate dataframes
for name, df in dfs.items():

        # Create a new row with the sum of all values above it
    sum_row = df.sum()
    sum_index = (name, 'Sum')
    sum_row.name = sum_index

        # Append the new row to the original DataFrame
    temp_1 = df.append(sum_row)
    df_sec_company_count_sum.append(temp_1)
# df_sec_company_count_all = pd.concat(df_sec_company_count_sum)
df_sec_company_count_sum_pd = pd.concat(df_sec_company_count_sum)

In [596]:
book = load_workbook(quantile_path)
writer = pd.ExcelWriter(quantile_path, engine = 'openpyxl')
writer.book = book
df_sec_company_count_sum_pd.to_excel(writer, sheet_name='quantile sec no', index=True)
writer.close()

In [597]:
# sort_companies_data['date'] = sort_companies_data['date'].dt.strftime(
#     '%Y-%m')

all_sort_companies_data = all_sort_companies_data.sort_values(
    by=["date", "quantile_rank"])

all_sort_companies_data.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
0,316,KOGAN.COM LTD,KGN AU Equity,4,2017,2017-12-31,21.245348,0.0,0.000000,63.576160,...,0.450545,4.396070,39.234862,-0.100562,-0.585575,6.093618,13.342381,0.383596,10.329886,0.0
1,72,INTEGRATED RESEARCH LTD,IRI AU Equity,8,2017,2017-12-31,29.162056,0.0,6.197098,81.095726,...,2.143583,-1.911082,18.358963,0.220855,-0.192501,-6.959629,33.721197,0.824675,14.484777,0.0
2,63,IONEER LTD,INR AU Equity,2,2017,2017-12-31,17.201488,0.0,5.350665,46.146900,...,-10.250857,-2.478196,45.795635,-0.169725,-0.408928,-9.662817,37.771376,-0.626318,8.396280,0.0
3,293,IPH LTD,IPH AU Equity,3,2017,2017-12-31,23.679710,0.0,7.315599,63.576160,...,3.226354,10.298060,30.841917,-0.229289,-0.332758,2.450075,10.992115,0.897637,11.673476,0.0
4,195,LIFESTYLE COMMUNITIES LTD,LIC AU Equity,11,2017,2017-12-31,26.838346,0.0,11.154776,69.205299,...,1.138423,-15.909984,51.098542,0.333638,-0.338132,7.912859,3.188926,1.407854,13.250107,0.0


**SRAY- Ret already calculated using excel by Dr May**
<br>**BLOOMBERG- Ret calculated using PYTHON CODE 2_portfolio.ipynb**

In [598]:

#set the query string
query_str = ''
query_str = 'quantile_rank == ' + str(quantile_rank-2)

sort_companies_data_top20per = all_sort_companies_data.query(query_str)

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp,cfp_sec_mean,cfp_sec_sd,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank
913,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,-5.514874,-19.624256,46.273294,0.304914,0.574319,5.738107,2.706188,1.271232,20.390770,3.0
914,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,-57.800481,-65.040082,199.914059,0.036214,-0.289501,-9.693295,23.544837,0.065891,26.205717,3.0
915,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,-2.927473,-27.039477,118.522743,0.203438,-0.142905,-9.197116,38.048781,0.434185,25.911848,3.0
916,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,-15.255438,-19.624256,46.273294,0.094413,0.126365,5.738107,2.706188,0.032183,21.886941,3.0
917,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,32.825535,11.913518,76.922930,0.271857,0.195212,7.387153,13.817713,-0.426493,25.190654,3.0


In [599]:
sort_companies_data_top20per.columns

Index(['firm no', 'firm_name', 'tkr', 'sec_no', 'year', 'date', 'esg', 'e',
       's', 'g', 'es', 'eg', 'sg', 'esg_cubic', 'px', 'fcf', 'pe', 'beta',
       'roa', 'vol_180d', 'sales', 'ret', 'rfr', 'rm', 'historical_market_cap',
       'ep', 'ep_sec_mean', 'ep_sec_sd', 'ep_z', 'sp', 'sp_sec_mean',
       'sp_sec_sd', 'sp_z', 'cfp', 'cfp_sec_mean', 'cfp_sec_sd', 'cfp_z',
       'value', 'roa_sec_mean', 'roa_sec_sd', 'roa_z', 'esg_value',
       'quantile_rank'],
      dtype='object')

In [600]:
companies = portfolio_selection_sorted.iloc[:, 0].drop_duplicates().tolist()
print(companies)

['29METALS LTD', 'A2B AUSTRALIA LTD', 'ABACUS PROPERTY GROUP', 'ACCENT GROUP LTD', 'ACCENT RESOURCES NL', 'ACRUX LTD', 'ADAIRS LTD', 'AGL ENERGY LTD', 'AIC MINES LTD', 'AINSWORTH GAME TECHNOLOGY LT', 'AJ LUCAS GROUP LTD', 'ALEXIUM INTERNATIONAL GROUP', 'ALKANE RESOURCES LTD', 'ALLKEM LTD', 'ALS LTD', 'ALTERITY THERAPEUTICS LTD', 'ALTIUM LTD', 'ALUMINA LTD', 'AMA GROUP LTD', 'ANSELL LTD', 'ANZ GROUP HOLDINGS LTD', 'APA GROUP', 'APM HUMAN SERVICES INTERNATI', 'ARA US HOSPITALITY TRUST', 'ARAFURA RARE EARTHS LTD', 'ARB CORP LTD', 'ARDENT LEISURE GROUP LTD', 'ARENA REIT', 'ARGENT MINERALS LTD', 'ARISTOCRAT LEISURE LTD', 'ASPEN GROUP', 'ASX LTD', 'ATLAS ARTERIA', 'AUB GROUP LTD', 'AUDINATE GROUP LTD', 'AURELIA METALS LTD', 'AURIZON HOLDINGS LTD', 'AUSSIE BROADBAND LTD', 'AUSTAL LTD', 'AUSTIN ENGINEERING LTD', 'AUSTRALIAN AGRICULTURAL CO', 'AUSTRALIAN ETHICAL INVT LTD', 'AUSTRALIAN FINANCE GROUP LTD', 'AUSTRALIAN VINTAGE LTD', 'AVECHO BIOTECHNOLOGY LTD', 'BABY BUNTING GROUP LTD', 'BANK OF QU

In [601]:
companies_sec=[]
initial_year=2017
stop_year=2023

company_dict1 = {}
company_dict_all = {}
sum_esg_sray = {}
sum_ret_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =0 and save it into company dictionary 
for year in range(initial_year,stop_year+1,1):    

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        each_year_companies_all_quantile = all_sort_companies_data.loc[(
            all_sort_companies_data['year'] == year)]

        sum = each_year_companies[indicator].sum()
        sum_esg_sray.setdefault(year, sum)

        sum_ret = each_year_companies['ret'].sum()
        sum_ret_dict.setdefault(year, sum_ret)

        companies = each_year_companies.iloc[:, 0]
        companies= companies.tolist() 

        total_companies = (each_year_companies_all_quantile.iloc[:, 0]).tolist()


        for c in range (len(companies)):
            company_dict1.setdefault(year,[]).append(companies[c])
            #company_dict2.setdefault(start_date,[]).append(companies[c])
        for c_all in range(len(total_companies)):
            company_dict_all.setdefault(
                year, []).append(total_companies[c_all])    
        

#make sure the list is correct     
#print(company_dict)
# print(f"company_dict1: {company_dict1}")
print(len(company_dict1))
print(f"sum_esg_sray_dic: {sum_esg_sray}")
print(f"sum_ret_dict: {sum_ret_dict}")
# print(len(company_dict2))


4
sum_esg_sray_dic: {2017: 0.0, 2018: 0.0, 2019: 2516.460256576538, 2020: 2741.256404876709, 2021: 1478.3147087097168, 2022: 1782.0900039672852, 2023: 0.0}
sum_ret_dict: {2017: 0.0, 2018: 0.0, 2019: 9.36851298253337, 2020: 2.2185156372620334, 2021: 4.006223389967872, 2022: -4.305886952069626, 2023: 0.0}


If it is equal-weighted, compute the mean of the stocks returns in the portfolio. 
$μ_(p,t)=  (\sum ret_{(i,t)} )/n$; i = 1, 2, 3, …, n
Where $ret_(i,t) = return-of-stock-i-at-time-t$, n = number of stocks in the portfolio


In [602]:
no_of_companies = {}
equal_weight_dict = {}
for date_range, company_names in company_dict1.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    #equal weightage
    # equal_weight = np.repeat(1/len(v), len(v))
    equal_weight = 1/len(company_names)
        # print(f"factor_weight: {factor_weight}")

    equal_weight_dict.setdefault(date_range, float(equal_weight))
    no_of_companies.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"equal_weight: {equal_weight_dict}")
print(f"no_of_companies: {no_of_companies}")


equal_weight: {2019: 0.016129032258064516, 2020: 0.015625, 2021: 0.015384615384615385, 2022: 0.014925373134328358}
no_of_companies: {2019: 62.0, 2020: 64.0, 2021: 65.0, 2022: 67.0}


### total companies from all quantile

In [603]:
no_of_companies_all = {}
for date_range, company_names in company_dict_all.items():

    # date = datetime.strptime(date_range, '%Y-%m')

    no_of_companies_all.setdefault(date_range, float(len(company_names)))

    # factor_weight_dict.setdefault(date_range, factor_weight)
print(f"no_of_companies: {no_of_companies_all}")


no_of_companies: {2017: 334.0, 2018: 334.0, 2019: 334.0, 2020: 334.0, 2021: 334.0, 2022: 334.0, 2023: 334.0}


## Calculate the number of companies (top 20%) based on the date

In [604]:
no_of_companies_items = no_of_companies.items()
no_of_companies_list = list(no_of_companies_items)

no_of_companies_df = pd.DataFrame(no_of_companies_list)
no_of_companies_df.columns = ['year', 'no of firms(top 20)']
no_of_companies_df.head()

,year,no of firms(top 20)
0,2019,62.0
1,2020,64.0
2,2021,65.0
3,2022,67.0


## Calculate the number of companies based on the date

In [605]:
no_of_companies_all_items = no_of_companies_all.items()
no_of_companies_all_list = list(no_of_companies_all_items)

no_of_companies_all_df = pd.DataFrame(no_of_companies_all_list)
no_of_companies_all_df.columns = ['year', 'no of firms']
no_of_companies_all_df.head()

,year,no of firms
0,2017,334.0
1,2018,334.0
2,2019,334.0
3,2020,334.0
4,2021,334.0


### Make the equal weight item into dataframe
Purpose: use for merging later on

In [606]:
equal_weight_items = equal_weight_dict.items()
equal_weight_list = list(equal_weight_items)

equal_weight_df = pd.DataFrame(equal_weight_list)
equal_weight_df.columns = ['year', 'eq_wg']
equal_weight_df.head()


,year,eq_wg
0,2019,0.016129
1,2020,0.015625
2,2021,0.015385
3,2022,0.014925


In [607]:
# no_of_companies_df.to_excel("eq_wg_no_of_companies_top20.xlsx")

In [608]:
sum_ret_items = sum_ret_dict.items()
sum_ret_list = list(sum_ret_items)

sum_ret_df = pd.DataFrame(sum_ret_list)
sum_ret_df.columns = ['year', 'sum_ret']
sum_ret_df.replace(0, np.nan, inplace=True)
sum_ret_df.head()


,year,sum_ret
0,2017,NaN
1,2018,NaN
2,2019,9.368513
3,2020,2.218516
4,2021,4.006223


### Make the sum of the esg_sray into dataframe
Purpose: merge into complete dataframe to make the data to be able to vefiry

In [609]:
sum_esg_sray_dic_items = sum_esg_sray.items()
sum_esg_sray_list = list(sum_esg_sray_dic_items)

sum_esg_sray_df = pd.DataFrame(sum_esg_sray_list)
sum_esg_sray_df.columns = ['year', 'sum_'+indicator]
sum_esg_sray_df.head()


,year,sum_e
0,2017,0.000000
1,2018,0.000000
2,2019,2516.460257
3,2020,2741.256405
4,2021,1478.314709


### Merging with the data of top 20% **only**

In [610]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    equal_weight_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_esg_sray_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    sum_ret_df, on='year')
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,cfp_z,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_e,sum_ret
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,0.304914,0.574319,5.738107,2.706188,1.271232,20.390770,3.0,0.016129,2516.460257,9.368513
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,0.036214,-0.289501,-9.693295,23.544837,0.065891,26.205717,3.0,0.016129,2516.460257,9.368513
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,0.203438,-0.142905,-9.197116,38.048781,0.434185,25.911848,3.0,0.016129,2516.460257,9.368513
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,0.094413,0.126365,5.738107,2.706188,0.032183,21.886941,3.0,0.016129,2516.460257,9.368513
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,0.271857,0.195212,7.387153,13.817713,-0.426493,25.190654,3.0,0.016129,2516.460257,9.368513


### Calculate the factor weight

In [611]:
if sort_companies_data_top20per[indicator].empty and sort_companies_data_top20per['sum_'+indicator].empty:
    sort_companies_data_top20per['fc_wg'] = None
else:
    sort_companies_data_top20per['fc_wg'] = sort_companies_data_top20per[indicator] / \
        sort_companies_data_top20per['sum_'+indicator]

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,value,roa_sec_mean,roa_sec_sd,roa_z,esg_value,quantile_rank,eq_wg,sum_e,sum_ret,fc_wg
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,0.574319,5.738107,2.706188,1.271232,20.390770,3.0,0.016129,2516.460257,9.368513,0.009590
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,-0.289501,-9.693295,23.544837,0.065891,26.205717,3.0,0.016129,2516.460257,9.368513,0.009602
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,-0.142905,-9.197116,38.048781,0.434185,25.911848,3.0,0.016129,2516.460257,9.368513,0.009626
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,0.126365,5.738107,2.706188,0.032183,21.886941,3.0,0.016129,2516.460257,9.368513,0.010010
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,0.195212,7.387153,13.817713,-0.426493,25.190654,3.0,0.016129,2516.460257,9.368513,0.010082


In [612]:
sort_companies_data_top20per['r'] = (
    sort_companies_data_top20per['ret'] - sort_companies_data_top20per['rfr'])

"""
r = ret -rfr
"""
sort_companies_data_top20per['eq_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_r'] = (
    sort_companies_data_top20per['r'] * sort_companies_data_top20per['fc_wg'])

"""
ret
"""
sort_companies_data_top20per['eq_wg_ret'] = (
    sort_companies_data_top20per['ret'] * sort_companies_data_top20per['eq_wg'])
sort_companies_data_top20per['fc_wg_ret'] = (
    sort_companies_data_top20per['ret'] * sort_companies_data_top20per['fc_wg'])

sort_companies_data_top20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,quantile_rank,eq_wg,sum_e,sum_ret,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,3.0,0.016129,2516.460257,9.368513,0.009590,0.133618,0.002155,0.001281,0.002322,0.001381
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,3.0,0.016129,2516.460257,9.368513,0.009602,-0.510060,-0.008227,-0.004897,-0.008060,-0.004798
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,3.0,0.016129,2516.460257,9.368513,0.009626,-0.447672,-0.007221,-0.004309,-0.007053,-0.004209
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,3.0,0.016129,2516.460257,9.368513,0.010010,0.109638,0.001768,0.001097,0.001935,0.001201
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,3.0,0.016129,2516.460257,9.368513,0.010082,0.258392,0.004168,0.002605,0.004335,0.002709
5,191,RAMSAY HEALTH CARE LTD,RHC AU Equity,6,2019,2019-12-31,39.010159,25.490788,15.296252,76.098740,...,3.0,0.016129,2516.460257,9.368513,0.010130,0.385972,0.006225,0.003910,0.006392,0.004015
6,116,GWA GROUP LTD,GWA AU Equity,3,2019,2019-12-31,42.842773,25.823013,23.972189,78.597229,...,3.0,0.016129,2516.460257,9.368513,0.010262,0.268210,0.004326,0.002752,0.004493,0.002859
7,231,NEW HOPE CORP LTD,NHC AU Equity,1,2019,2019-12-31,40.177044,26.245848,28.113665,66.074654,...,3.0,0.016129,2516.460257,9.368513,0.010430,-0.679409,-0.010958,-0.007086,-0.010791,-0.006978
8,314,RELIANCE WORLDWIDE CORP LTD,RWC AU Equity,3,2019,2019-12-31,35.781109,27.212322,17.019346,63.004215,...,3.0,0.016129,2516.460257,9.368513,0.010814,-0.074810,-0.001207,-0.000809,-0.001040,-0.000697
9,333,COLES GROUP LTD,COL AU Equity,5,2019,2019-12-31,49.220402,28.239202,32.496979,86.785072,...,3.0,0.016129,2516.460257,9.368513,0.011222,0.140201,0.002261,0.001573,0.002428,0.001690


## Excess return portfolio
### Equal Weight and Factor Weight

In [613]:
initial_year = 2017
stop_year = 2023

eq_wg_r_p_dict = {}
fc_wg_r_p_dict = {}
eq_wg_ret_p_dict = {}
fc_wg_ret_p_dict = {}
# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        r
        """
        eq_wg_r_p = each_year_companies['eq_wg_r'].sum()
        fc_wg_r_p = each_year_companies['fc_wg_r'].sum()


        """
        ret
        """
        eq_wg_ret_p = each_year_companies['eq_wg_ret'].sum()
        fc_wg_ret_p = each_year_companies['fc_wg_ret'].sum()


        eq_wg_r_p_dict.setdefault(year, eq_wg_r_p)
        fc_wg_r_p_dict.setdefault(year, fc_wg_r_p)

        eq_wg_ret_p_dict.setdefault(year, eq_wg_ret_p)
        fc_wg_ret_p_dict.setdefault(year, fc_wg_ret_p)


print(f"eq_wg_r_p_dict: {eq_wg_r_p_dict}")
print(f"fc_wg_r_p_dict: {fc_wg_r_p_dict}")
print(f"eq_wg_ret_p_dict: {eq_wg_ret_p_dict}")
print(f"fc_wg_ret_p_dict: {fc_wg_ret_p_dict}")
# print(len(company_dict2))


eq_wg_r_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.14091116100860274, 2020: 0.033989025582219265, 2021: 0.06106320599950572, 2022: 0.0, 2023: 0.0}
fc_wg_r_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.13977946476990843, 2020: 0.03042095702172134, 2021: 0.054395293906378775, 2022: 0.0, 2023: 0.0}
eq_wg_ret_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.15110504810537692, 2020: 0.03466430683221927, 2021: 0.06163420599950572, 2022: -0.064266969433875, 2023: 0.0}
fc_wg_ret_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.14990581274392184, 2020: 0.031092611660983024, 2021: 0.05496629390637876, 2022: -0.0710659559744208, 2023: 0.0}


In [614]:
eq_wg_r_p_dict_items = eq_wg_r_p_dict.items()
eq_wg_r_p_dict_list = list(eq_wg_r_p_dict_items)

eq_wg_r_p_dict_df = pd.DataFrame(eq_wg_r_p_dict_list)
eq_wg_r_p_dict_df.columns = ['year', 'eq_wg_r_p']
eq_wg_r_p_dict_df.replace(0, np.nan, inplace = True)
eq_wg_r_p_dict_df.head()

,year,eq_wg_r_p
0,2017,NaN
1,2018,NaN
2,2019,0.140911
3,2020,0.033989
4,2021,0.061063


In [615]:
fc_wg_r_p_dict_items = fc_wg_r_p_dict.items()
fc_wg_r_p_dict_list = list(fc_wg_r_p_dict_items)

fc_wg_r_p_dict_df = pd.DataFrame(fc_wg_r_p_dict_list)
fc_wg_r_p_dict_df.columns = ['year', 'fc_wg_r_p']
fc_wg_r_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_r_p_dict_df.head()


,year,fc_wg_r_p
0,2017,NaN
1,2018,NaN
2,2019,0.139779
3,2020,0.030421
4,2021,0.054395


In [616]:
eq_wg_ret_p_dict_items = eq_wg_ret_p_dict.items()
eq_wg_ret_p_dict_list = list(eq_wg_ret_p_dict_items)

eq_wg_ret_p_dict_df = pd.DataFrame(eq_wg_ret_p_dict_list)
eq_wg_ret_p_dict_df.columns = ['year', 'eq_wg_ret_p']
eq_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_ret_p_dict_df.head()


,year,eq_wg_ret_p
0,2017,NaN
1,2018,NaN
2,2019,0.151105
3,2020,0.034664
4,2021,0.061634


In [617]:
fc_wg_ret_p_dict_items = fc_wg_ret_p_dict.items()
fc_wg_ret_p_dict_list = list(fc_wg_ret_p_dict_items)

fc_wg_ret_p_dict_df = pd.DataFrame(fc_wg_ret_p_dict_list)
fc_wg_ret_p_dict_df.columns = ['year', 'fc_wg_ret_p']
fc_wg_ret_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_ret_p_dict_df.head()


,year,fc_wg_ret_p
0,2017,NaN
1,2018,NaN
2,2019,0.149906
3,2020,0.031093
4,2021,0.054966


In [618]:
SG_esg_int = pd.merge(no_of_companies_all_df, no_of_companies_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_r_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, eq_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_ret_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,2017,334.0,NaN,NaN,NaN,NaN,NaN
1,2018,334.0,NaN,NaN,NaN,NaN,NaN
2,2019,334.0,62.0,0.140911,0.139779,0.151105,0.149906
3,2020,334.0,64.0,0.033989,0.030421,0.034664,0.031093
4,2021,334.0,65.0,0.061063,0.054395,0.061634,0.054966


### Merging the eq and fc wg of the r_p into the big dataframe

In [619]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_r_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_r_p_dict_df, on='year')

sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_ret_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_ret_p_dict_df, on='year')

sort_companies_data_top20per.head(15)


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_wg,r,eq_wg_r,fc_wg_r,eq_wg_ret,fc_wg_ret,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,0.009590,0.133618,0.002155,0.001281,0.002322,0.001381,0.140911,0.139779,0.151105,0.149906
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,0.009602,-0.510060,-0.008227,-0.004897,-0.008060,-0.004798,0.140911,0.139779,0.151105,0.149906
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,0.009626,-0.447672,-0.007221,-0.004309,-0.007053,-0.004209,0.140911,0.139779,0.151105,0.149906
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,0.010010,0.109638,0.001768,0.001097,0.001935,0.001201,0.140911,0.139779,0.151105,0.149906
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,0.010082,0.258392,0.004168,0.002605,0.004335,0.002709,0.140911,0.139779,0.151105,0.149906
5,191,RAMSAY HEALTH CARE LTD,RHC AU Equity,6,2019,2019-12-31,39.010159,25.490788,15.296252,76.098740,...,0.010130,0.385972,0.006225,0.003910,0.006392,0.004015,0.140911,0.139779,0.151105,0.149906
6,116,GWA GROUP LTD,GWA AU Equity,3,2019,2019-12-31,42.842773,25.823013,23.972189,78.597229,...,0.010262,0.268210,0.004326,0.002752,0.004493,0.002859,0.140911,0.139779,0.151105,0.149906
7,231,NEW HOPE CORP LTD,NHC AU Equity,1,2019,2019-12-31,40.177044,26.245848,28.113665,66.074654,...,0.010430,-0.679409,-0.010958,-0.007086,-0.010791,-0.006978,0.140911,0.139779,0.151105,0.149906
8,314,RELIANCE WORLDWIDE CORP LTD,RWC AU Equity,3,2019,2019-12-31,35.781109,27.212322,17.019346,63.004215,...,0.010814,-0.074810,-0.001207,-0.000809,-0.001040,-0.000697,0.140911,0.139779,0.151105,0.149906
9,333,COLES GROUP LTD,COL AU Equity,5,2019,2019-12-31,49.220402,28.239202,32.496979,86.785072,...,0.011222,0.140201,0.002261,0.001573,0.002428,0.001690,0.140911,0.139779,0.151105,0.149906


In [620]:
sort_companies_data_top20per['rm_r_y1'] = (
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']) * 12
sort_companies_data_top20per['rm_r_y2'] = (((1+(
    sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr'])) ** 12) - 1)

sort_companies_data_top20per['eq_wg_r_p_y1'] = sort_companies_data_top20per['eq_wg_r_p'] *12
sort_companies_data_top20per['eq_wg_r_p_y2'] = (((1 + sort_companies_data_top20per['eq_wg_r_p'])**12) - 1)
sort_companies_data_top20per['eq_r_alpha_y1'] = sort_companies_data_top20per['eq_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_r_alpha_y2'] = sort_companies_data_top20per['eq_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,0.140911,0.139779,0.151105,0.149906,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,0.140911,0.139779,0.151105,0.149906,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,0.140911,0.139779,0.151105,0.149906,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,0.140911,0.139779,0.151105,0.149906,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,0.140911,0.139779,0.151105,0.149906,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819


In [621]:
sort_companies_data_top20per['fc_wg_r_p_y1'] = sort_companies_data_top20per['fc_wg_r_p'] * 12
sort_companies_data_top20per['fc_wg_r_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_r_p'])**12) - 1)
sort_companies_data_top20per['fc_r_alpha_y1'] = sort_companies_data_top20per['fc_wg_r_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_r_alpha_y2'] = sort_companies_data_top20per['fc_wg_r_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,1.03893,1.708499,1.690934,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233


In [622]:

# sort_companies_data_top20per.to_excel(
#     "../3.04/SG_esg_top20.xlsx", sheet_name="main")

## CAPM-adjusted return ($r_{c}$)

$r_(c,t)=r_(f,t)+β_(p,t) (r_(m,t)-r_(f,t))$

In [623]:
sort_companies_data_top20per['rc'] = sort_companies_data_top20per['rfr'] + \
    (sort_companies_data_top20per['beta'] *
    (sort_companies_data_top20per['rm'] - sort_companies_data_top20per['rfr']))
    
"""
equal weight rc
"""
sort_companies_data_top20per['eq_wg_rc_int'] = sort_companies_data_top20per ['eq_wg'] * sort_companies_data_top20per['rc']

"""
factor weight rc
"""
sort_companies_data_top20per['fc_wg_rc_int'] = sort_companies_data_top20per['fc_wg'] * \
    sort_companies_data_top20per['rc']

sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_r_p_y2,eq_r_alpha_y1,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233,0.068074,0.001098,0.000653
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233,0.002370,0.000038,0.000023
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233,0.000612,0.000010,0.000006
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233,0.034239,0.000552,0.000343
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,3.864318,0.652004,2.155819,1.677354,3.806732,0.638423,2.098233,0.117371,0.001893,0.001183


In [624]:
initial_year = 2017
stop_year = 2023

eq_wg_rc_p_dict = {}
fc_wg_rc_p_dict = {}

# company_dict2 = {}

#loop year, loop every months to get the companies quantile rank =4 and save it into company dictionary
for year in range(initial_year, stop_year+1, 1):

        each_year_companies = sort_companies_data_top20per.loc[(
            sort_companies_data_top20per['year'] == year)]

        """
        rc
        """
        eq_wg_rc_p = each_year_companies['eq_wg_rc_int'].sum()
        fc_wg_rc_p = each_year_companies['fc_wg_rc_int'].sum()

        eq_wg_rc_p_dict.setdefault(year, eq_wg_rc_p)
        fc_wg_rc_p_dict.setdefault(year, fc_wg_rc_p)


print(f"eq_wg_rc_p_dict: {eq_wg_rc_p_dict}")
print(f"fc_wg_rc_p_dict: {fc_wg_rc_p_dict}")


eq_wg_rc_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.08363216441535702, 2020: -0.10510172729326414, 2021: 0.15581478432894622, 2022: 0.0, 2023: 0.0}
fc_wg_rc_p_dict: {2017: 0.0, 2018: 0.0, 2019: 0.08586797262319354, 2020: -0.10507542251912702, 2021: 0.15689874189830366, 2022: 0.0, 2023: 0.0}


In [625]:
eq_wg_rc_p_dict_items = eq_wg_rc_p_dict.items()
eq_wg_rc_p_dict_list = list(eq_wg_rc_p_dict_items)

eq_wg_rc_p_dict_df = pd.DataFrame(eq_wg_rc_p_dict_list)
eq_wg_rc_p_dict_df.columns = ['year', 'eq_wg_rc_p']
eq_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
eq_wg_rc_p_dict_df.head()


,year,eq_wg_rc_p
0,2017,NaN
1,2018,NaN
2,2019,0.083632
3,2020,-0.105102
4,2021,0.155815


In [626]:
fc_wg_rc_p_dict_items = fc_wg_rc_p_dict.items()
fc_wg_rc_p_dict_list = list(fc_wg_rc_p_dict_items)

fc_wg_rc_p_dict_df = pd.DataFrame(fc_wg_rc_p_dict_list).dropna()
fc_wg_rc_p_dict_df.columns = ['year', 'fc_wg_rc_p']
fc_wg_rc_p_dict_df.replace(0, np.nan, inplace=True)
fc_wg_rc_p_dict_df.head()


,year,fc_wg_rc_p
0,2017,NaN
1,2018,NaN
2,2019,0.085868
3,2020,-0.105075
4,2021,0.156899


In [627]:
SG_esg_int = pd.merge(SG_esg_int, eq_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int = pd.merge(SG_esg_int, fc_wg_rc_p_dict_df,
                        on="year", how="left")
SG_esg_int.head()

,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p
0,2017,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,334.0,62.0,0.140911,0.139779,0.151105,0.149906,0.083632,0.085868
3,2020,334.0,64.0,0.033989,0.030421,0.034664,0.031093,-0.105102,-0.105075
4,2021,334.0,65.0,0.061063,0.054395,0.061634,0.054966,0.155815,0.156899


In [628]:
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    eq_wg_rc_p_dict_df, on='year')
sort_companies_data_top20per = sort_companies_data_top20per.merge(
    fc_wg_rc_p_dict_df, on='year')

sort_companies_data_top20per.head(15)

,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_r_alpha_y2,fc_wg_r_p_y1,fc_wg_r_p_y2,fc_r_alpha_y1,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.068074,0.001098,0.000653,0.083632,0.085868
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.002370,0.000038,0.000023,0.083632,0.085868
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.000612,0.000010,0.000006,0.083632,0.085868
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.034239,0.000552,0.000343,0.083632,0.085868
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.117371,0.001893,0.001183,0.083632,0.085868
5,191,RAMSAY HEALTH CARE LTD,RHC AU Equity,6,2019,2019-12-31,39.010159,25.490788,15.296252,76.098740,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.079796,0.001287,0.000808,0.083632,0.085868
6,116,GWA GROUP LTD,GWA AU Equity,3,2019,2019-12-31,42.842773,25.823013,23.972189,78.597229,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.119674,0.001930,0.001228,0.083632,0.085868
7,231,NEW HOPE CORP LTD,NHC AU Equity,1,2019,2019-12-31,40.177044,26.245848,28.113665,66.074654,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.070931,0.001144,0.000740,0.083632,0.085868
8,314,RELIANCE WORLDWIDE CORP LTD,RWC AU Equity,3,2019,2019-12-31,35.781109,27.212322,17.019346,63.004215,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.047711,0.000770,0.000516,0.083632,0.085868
9,333,COLES GROUP LTD,COL AU Equity,5,2019,2019-12-31,49.220402,28.239202,32.496979,86.785072,...,2.155819,1.677354,3.806732,0.638423,2.098233,0.026975,0.000435,0.000303,0.083632,0.085868


In [629]:
sort_companies_data_top20per['eq_wg_rc_p_y1'] = sort_companies_data_top20per['eq_wg_rc_p'] * 12
sort_companies_data_top20per['eq_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['eq_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['eq_rc_alpha_y1'] = sort_companies_data_top20per['eq_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['eq_rc_alpha_y2'] = sort_companies_data_top20per['eq_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,fc_r_alpha_y2,rc,eq_wg_rc_int,fc_wg_rc_int,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,2.098233,0.068074,0.001098,0.000653,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,2.098233,0.002370,0.000038,0.000023,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,2.098233,0.000612,0.000010,0.000006,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,2.098233,0.034239,0.000552,0.000343,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,2.098233,0.117371,0.001893,0.001183,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801


In [630]:
sort_companies_data_top20per['fc_wg_rc_p_y1'] = sort_companies_data_top20per['fc_wg_rc_p'] * 12
sort_companies_data_top20per['fc_wg_rc_p_y2'] = ((
    (1 + sort_companies_data_top20per['fc_wg_rc_p'])**12) - 1)
sort_companies_data_top20per['fc_rc_alpha_y1'] = sort_companies_data_top20per['fc_wg_rc_p_y1'] - \
    sort_companies_data_top20per['rm_r_y1']
sort_companies_data_top20per['fc_rc_alpha_y2'] = sort_companies_data_top20per['fc_wg_rc_p_y2'] - \
    sort_companies_data_top20per['rm_r_y2']
sort_companies_data_top20per.head()


,firm no,firm_name,tkr,sec_no,year,date,esg,e,s,g,...,eq_wg_rc_p,fc_wg_rc_p,eq_wg_rc_p_y1,eq_wg_rc_p_y2,eq_rc_alpha_y1,eq_rc_alpha_y2,fc_wg_rc_p_y1,fc_wg_rc_p_y2,fc_rc_alpha_y1,fc_rc_alpha_y2
0,54,GROWTHPOINT PROPERTIES AUSTR,GOZ AU Equity,11,2019,2019-12-31,40.207222,24.131681,17.623941,78.717644,...,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801,1.030416,1.68735,-0.008515,-0.021149
1,70,PALADIN ENERGY LTD,PDN AU Equity,1,2019,2019-12-31,52.700935,24.161884,55.259975,78.597229,...,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801,1.030416,1.68735,-0.008515,-0.021149
2,3,ST BARBARA LTD,SBM AU Equity,2,2019,2019-12-31,51.966602,24.222288,68.077385,63.576160,...,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801,1.030416,1.68735,-0.008515,-0.021149
3,277,DEXUS INDUSTRIA REIT,DXI AU Equity,11,2019,2019-12-31,43.647518,25.188765,27.025393,78.597229,...,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801,1.030416,1.68735,-0.008515,-0.021149
4,48,MACQUARIE GROUP LTD,MQG AU Equity,7,2019,2019-12-31,50.186096,25.369980,33.796856,91.240219,...,0.083632,0.085868,1.003586,1.621698,-0.035344,-0.086801,1.030416,1.68735,-0.008515,-0.021149


In [631]:
SG_int_wanted = sort_companies_data_top20per[[
    'year', 'rm_r_y1', 'rm_r_y2', 'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_r_alpha_y2', 'eq_wg_r_p_y2', 'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_r_alpha_y2', 'fc_wg_r_p_y2', 'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_rc_alpha_y2', 'eq_wg_rc_p_y2', 'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_rc_alpha_y2', 'fc_wg_rc_p_y2']]
SG_int_wanted = SG_int_wanted.drop_duplicates()
SG_int_wanted.head()


,year,rm_r_y1,rm_r_y2,eq_wg_r_p_y1,eq_r_alpha_y1,eq_r_alpha_y2,eq_wg_r_p_y2,fc_wg_r_p_y1,fc_r_alpha_y1,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2019,1.038930,1.708499,1.690934,0.652004,2.155819,3.864318,1.677354,0.638423,2.098233,3.806732,1.003586,-0.035344,-0.086801,1.621698,1.030416,-0.008515,-0.021149,1.687350
62,2020,-1.143442,-0.699302,0.407868,1.551310,1.192753,0.493452,0.365051,1.508493,1.132071,0.432769,-1.261221,-0.117779,-0.036893,-0.736194,-1.260905,-0.117463,-0.036800,-0.736101
126,2021,1.629852,3.610187,0.732758,-0.897093,-2.573638,1.036550,0.652744,-0.977108,-2.722016,0.888172,1.869777,0.239926,1.073876,4.684064,1.882785,0.252933,1.138176,4.748363
191,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [632]:
SG_esg_int = pd.merge(SG_esg_int, SG_int_wanted,
                        on="year", how="left")
SG_esg_int.head()


,year,no of firms,no of firms(top 20),eq_wg_r_p,fc_wg_r_p,eq_wg_ret_p,fc_wg_ret_p,eq_wg_rc_p,fc_wg_rc_p,rm_r_y1,...,fc_r_alpha_y2,fc_wg_r_p_y2,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_rc_alpha_y2,eq_wg_rc_p_y2,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_rc_alpha_y2,fc_wg_rc_p_y2
0,2017,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,334.0,62.0,0.140911,0.139779,0.151105,0.149906,0.083632,0.085868,1.038930,...,2.098233,3.806732,1.003586,-0.035344,-0.086801,1.621698,1.030416,-0.008515,-0.021149,1.687350
3,2020,334.0,64.0,0.033989,0.030421,0.034664,0.031093,-0.105102,-0.105075,-1.143442,...,1.132071,0.432769,-1.261221,-0.117779,-0.036893,-0.736194,-1.260905,-0.117463,-0.036800,-0.736101
4,2021,334.0,65.0,0.061063,0.054395,0.061634,0.054966,0.155815,0.156899,1.629852,...,-2.722016,0.888172,1.869777,0.239926,1.073876,4.684064,1.882785,0.252933,1.138176,4.748363


In [633]:
SG_esg_int = SG_esg_int[['year', 'no of firms', 'no of firms(top 20)',
                            'rm_r_y1', 'rm_r_y2', 'eq_wg_ret_p', 'fc_wg_ret_p', 'eq_wg_r_p',
                            'eq_wg_r_p_y1', 'eq_r_alpha_y1', 'eq_wg_r_p_y2', 'eq_r_alpha_y2', 'fc_wg_r_p',
                            'fc_wg_r_p_y1', 'fc_r_alpha_y1', 'fc_wg_r_p_y2', 'fc_r_alpha_y2', 'eq_wg_rc_p',
                            'eq_wg_rc_p_y1', 'eq_rc_alpha_y1', 'eq_wg_rc_p_y2', 'eq_rc_alpha_y2', 'fc_wg_rc_p',
                            'fc_wg_rc_p_y1', 'fc_rc_alpha_y1', 'fc_wg_rc_p_y2', 'fc_rc_alpha_y2']]
SG_esg_int.head()


,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,2017,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,334.0,62.0,1.038930,1.708499,0.151105,0.149906,0.140911,1.690934,0.652004,...,0.083632,1.003586,-0.035344,1.621698,-0.086801,0.085868,1.030416,-0.008515,1.687350,-0.021149
3,2020,334.0,64.0,-1.143442,-0.699302,0.034664,0.031093,0.033989,0.407868,1.551310,...,-0.105102,-1.261221,-0.117779,-0.736194,-0.036893,-0.105075,-1.260905,-0.117463,-0.736101,-0.036800
4,2021,334.0,65.0,1.629852,3.610187,0.061634,0.054966,0.061063,0.732758,-0.897093,...,0.155815,1.869777,0.239926,4.684064,1.073876,0.156899,1.882785,0.252933,4.748363,1.138176


## Average of the int_reporting

In [634]:
SG_esg_int_mean = SG_esg_int.mean(numeric_only=True)
SG_esg_int_mean = pd.DataFrame(SG_esg_int_mean).T
SG_esg_int_mean['year'] = SG_esg_int_mean['year'].replace([SG_esg_int_mean['year'].mean()], 'average')
SG_esg_int_mean = pd.concat([SG_esg_int, SG_esg_int_mean], axis=0)
SG_esg_int_mean.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
3,2020,334.0,64.0,-1.143442,-0.699302,0.034664,0.031093,0.033989,0.407868,1.551310,...,-0.105102,-1.261221,-0.117779,-0.736194,-0.036893,-0.105075,-1.260905,-0.117463,-0.736101,-0.036800
4,2021,334.0,65.0,1.629852,3.610187,0.061634,0.054966,0.061063,0.732758,-0.897093,...,0.155815,1.869777,0.239926,4.684064,1.073876,0.156899,1.882785,0.252933,4.748363,1.138176
5,2022,334.0,67.0,NaN,NaN,-0.064267,-0.071066,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,334.0,64.5,0.508447,1.539795,0.045784,0.041225,0.078654,0.943854,0.435407,...,0.044782,0.537381,0.028934,1.856522,0.316728,0.045897,0.550765,0.042318,1.899871,0.360076


## T-test of the int_reporting

a: an array of sample observations <br>
popmean: the expected population mean (The population mean is an average of a group characteristic.) <br>

In [635]:
SG_res_dict = {}
temp_list = []

"""
make the t-stat, p-value into dictionary
"""
for column in range(3, SG_esg_int.columns.__len__()):
    tscore, pvalue = ttest_1samp(
        a=SG_esg_int.dropna()[SG_esg_int.columns[column]], popmean=0)
    temp_dict = {SG_esg_int.columns[column]: {
        "t-score": tscore, "p-value": pvalue}}
    SG_res_dict = {**SG_res_dict, **temp_dict}

"""
make the dictionary into dataframe
"""
res_df = pd.DataFrame.from_dict({(i): SG_res_dict[i]
                                    for i in SG_res_dict.keys()},
                                orient='index').T
res_df.reset_index(inplace=True)
res_df.rename(columns={'index': 'year'}, inplace=True)
res_df
# SG_res_dict = {**temp_dict, **SG_res_dict}
# testing_df = pd.DataFrame(SG_res_dict, index=[0])
# testing_df
# SG_res_dict

,year,rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,eq_wg_r_p_y2,eq_r_alpha_y2,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
0,t-score,0.602871,1.234900,2.343460,2.167642,2.450713,2.450713,0.608925,1.720785,0.179009,...,0.575714,0.575714,0.267546,1.183182,0.836027,0.586757,0.586757,0.385051,1.197304,0.925464
1,p-value,0.607851,0.342261,0.143821,0.162484,0.133867,0.133867,0.604527,0.227431,0.874424,...,0.622954,0.622954,0.814114,0.358322,0.491110,0.616775,0.616775,0.737292,0.353850,0.452425


In [636]:
SG_esg_int = pd.concat([SG_esg_int_mean, res_df])
SG_esg_int.tail()

,year,no of firms,no of firms(top 20),rm_r_y1,rm_r_y2,eq_wg_ret_p,fc_wg_ret_p,eq_wg_r_p,eq_wg_r_p_y1,eq_r_alpha_y1,...,eq_wg_rc_p,eq_wg_rc_p_y1,eq_rc_alpha_y1,eq_wg_rc_p_y2,eq_rc_alpha_y2,fc_wg_rc_p,fc_wg_rc_p_y1,fc_rc_alpha_y1,fc_wg_rc_p_y2,fc_rc_alpha_y2
5,2022,334.0,67.0,NaN,NaN,-0.064267,-0.071066,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2023,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,average,334.0,64.5,0.508447,1.539795,0.045784,0.041225,0.078654,0.943854,0.435407,...,0.044782,0.537381,0.028934,1.856522,0.316728,0.045897,0.550765,0.042318,1.899871,0.360076
0,t-score,NaN,NaN,0.602871,1.234900,2.343460,2.167642,2.450713,2.450713,0.608925,...,0.575714,0.575714,0.267546,1.183182,0.836027,0.586757,0.586757,0.385051,1.197304,0.925464
1,p-value,NaN,NaN,0.607851,0.342261,0.143821,0.162484,0.133867,0.133867,0.604527,...,0.622954,0.622954,0.814114,0.358322,0.491110,0.616775,0.616775,0.737292,0.353850,0.452425


### Export the result into excel file

In [637]:
sort_companies_data_top20per.to_excel(top_path, sheet_name="main", index=False)
SG_esg_int.to_excel(reporting_path, sheet_name="main", index=False)
